In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
#import shutil
#import os
import json
import sources.REmodeling as mod

In [ ]:
class CONFIG: 
    def __init__(this,
            #EPS = 1e-8, 
            EPOCHS = 5, # 3~5
            BATCH_SIZE = 16, # 8, 32
            LEARNING_RATE = 3e-5, # 1e-5
            MAX_LENGTH = 128, 
            ENCODER_TYPE = 'bert-base', # large, RoBERTa, DeBERTa
            OPTIMIZER ='adamw',   
            DEVICE_TYPE = "cuda", # Cuda or alternative
            SAVE_MODEL=False,
            TASK='RE-classification'):
        this.EPOCHS=EPOCHS
        this.BATCH_SIZE=BATCH_SIZE
        this.LEARNING_RATE = LEARNING_RATE
        this.MAX_LENGTH = MAX_LENGTH
        this.ENCODER_TYPE = ENCODER_TYPE
        this.OPTIMIZER = OPTIMIZER
        this.DEVICE_TYPE = DEVICE_TYPE
        this.SAVE_MODEL=SAVE_MODEL
        this.TASK=TASK
    

In [ ]:
configs=[]
types=['bert-large','roberta-large']
for type in types:
    for eps in [3,5,8]:
        configs.append(CONFIG(ENCODER_TYPE=type,EPOCHS=eps))
    for lr in [1e-5,1e-4]:
        configs.append(CONFIG(ENCODER_TYPE=type,LEARNING_RATE=lr))
    if configs[-1].ENCODER_TYPE=='bert-large' and configs[-1].EPOCHS==5 and configs[-1].LEARNING_RATE==3e-5:
        configs[-1].SAVE_MODLE=True
    if configs[-1].ENCODER_TYPE=='roberta-large' and configs[-1].EPOCHS==5 and configs[-1].LEARNING_RATE==3e-5:
        configs[-1].SAVE_MODLE=True

In [ ]:

class Datapack:
    def __init__(this,train,test,val):
        this.test_ds,this.reladict,this.testx,this.testy=this.load_dataset(test)
        this.valid_ds,reladict,this.validx,this.validy=this.load_dataset(val,this.reladict,shuffle=True)
        this.train_ds,this.reladict,this.trainx,this.trainy=this.load_dataset(train,this.reladict,shuffle=True)
        this.invdict=np.zeros(len(this.reladict)).tolist()
        for k,v in reladict.items():
            this.invdict[v]=k
        
    def load_dataset(this,path,reladict={},shuffle=False):
        with open(path,'r',encoding='UTF-8') as f:
            data=[json.loads(line) for line in f]
        sent=[]
        relation=[]
        for d in data:
            token=d['token']
            for i in range(d['h']['pos'][0],d['h']['pos'][1]):
                token[i]='[MASK]'
            for i in range(d['t']['pos'][0],d['t']['pos'][1]):
                token[i]='[MASK]'
            sent.append([' '.join(token)])
            if reladict.get(d['relation'],False)==False:
                reladict.setdefault(d['relation'],len(reladict))
            relation.append(reladict[d['relation']])
        temp=np.zeros(shape=(len(relation),len(reladict)))
        #print(temp.shape)
        #print(reladict)
        for i in range(len(relation)):
            temp[i][relation[i]]=1
        if shuffle:
            data=tf.data.Dataset.from_tensor_slices((sent,temp)).shuffle(5000).batch(configs[0].BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
        else:
            data=tf.data.Dataset.from_tensor_slices((sent,temp)).batch(configs[0].BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
        return data,reladict,sent,relation


In [ ]:
gpus = tf.config.experimental.list_physical_devices("GPU")
for gpu in gpus:
    tf.config.experimental.set_memory_growth(gpu, True)
datapack=Datapack('semeval_train.txt','semeval_test.txt','semeval_val.txt')
#for text_batch, label_batch in train_ds.take(1):
  #for i in range(3):
    #print(f'Review: {text_batch.numpy()[i]}')
    #label = label_batch.numpy()[i]
    #print(f'Label : {label}')
    #print(len(label))

In [ ]:
if False:
    with open('results/NRE_test_result.json','w',encoding='UTF-8')as f:
        json.dump({},f,indent=2)
n=6

In [ ]:
config=CONFIG(ENCODER_TYPE='roberta-large')
model,result=mod.modeling(config,datapack)

In [ ]:
#for i in range(n,len(configs)):
#    model,result=mod.modeling(configs[i],datapack)
#    n=i+1

In [ ]:
print(result['Result']['Weighted'])
tf.keras.backend.clear_session()